# 🤖 MODELAGEM DE MACHINE LEARNING PARA SCORE DE CRÉDITO

**Objetivo:** Desenvolver, avaliar e selecionar modelos de Machine Learning para prever a inadimplência de clientes.

### 📌 Etapas
- Reparar dados para Machine Learning
- Codificar variáveis categóricas
- Dividir em treino e teste
- Treinar múltiplos modelos de classificação
- Avaliar performance com métricas adequadas
- Selecionar melhor modelo
- Analisar feature importance
- Salvar modelo para produção

### 🏆 Modelo que vamos testar
- Random Forest (Ensemble)
- KNN (K-Nearest Neighbors)
- Logistic Regression (Linear)
- Decision-Tree (Base do Random Forest)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # Para salvar modelos
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, ConfusionMatrixDisplay
)

# Configs de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
# Carregando dados do SQLite
DB_PATH = 'data/database.db'
print(f"\n📂 Carregando dados: {DB_PATH}")

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM clientes", conn)
conn.close()

In [ ]:
# Codificando variáveis categóricas
print("🔄 CODIFICANDO VARIÁVEIS CATEGÓRICAS")

# Identifica
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'inadimplente' in cat_cols:
  cat_cols.remove('inadimplente')

print(f"\n📝 Variáveis categóricas: {cat_cols}")

# Codifica e salva encoders
encoders = {}

os.makedirs('../models', exist_ok=True)

for col in cat_cols:
  encoder = LabelEncoder()
  df[col] = encoder.fit_transform(df[col])
  encoders[col] = encoder
    
  joblib.dump(encoder, f'../models/encoder_{col}.pkl')
  print(f"   ✅ {col}: {list(encoder.classes_)}")

print(f"\n💾 {len(encoders)} encoders salvos em ../models/")

In [ ]:
# Prepara X e Y
print("✂️ SEPARANDO FEATURES (X) E TARGET (y)")

# Remove ID (coluna desnecessária) e Target (Principal)
X = df.drop(columns=['id_cliente', 'inadimplente'])
y = df['inadimplente']

print(f"\n📊 X: {X.shape} - {X.shape[1]} features")
print(f"📊 y: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

In [ ]:
# Dividir Treino/Teste
print("🔀 TREINO/TESTE (70/30)")

X_train, X_test, y_train, y_test = train_test_split(
  X, y,
  test_size=0.3,
  random_state=42,
  stratify=y
)

print(f"\n✅ Treino: {len(X_train):,} ({len(X_train)/len(X)*100:.0f}%)")
print(f"✅ Teste:  {len(X_test):,} ({len(X_test)/len(X)*100:.0f}%)")
print(f"\nTaxa inadimplência treino: {y_train.mean()*100:.2f}%")
print(f"Taxa inadimplência teste:  {y_test.mean()*100:.2f}%")

In [ ]:
# Normalização (Para KNN e LR)
print("⚖️ NORMALIZAÇÃO (StandardScaler)")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converte de volta para DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

print("✅ Dados normalizados (média=0, std=1)")
joblib.dump(scaler, 'models/scaler.pkl')
print("💾 Scaler salvo: models/scaler.pkl")